In [120]:
### Hyperparams
batch_size = 1

In [121]:
from torch.utils.data import Dataset
import pandas as pd
from skimage import io
import os

class GZ2Dataset(Dataset):
    def __init__(self, csv_file_maps, csv_file_data, root_dir, transform=None):
        self.maps = pd.read_csv(csv_file_maps)
        self.data = pd.read_csv(csv_file_data)
        self.root_dir = root_dir
        self.transform = transform

        self.files = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.files[index])#str(self.maps.iloc[index,2]) + '.jpg')
        image = io.imread(img_path)
        
        dr7objid = self.maps.loc[self.maps['asset_id'] == int(self.files[index][:-4])]['objid'].values[0]

        image_data = self.data.loc[self.data['dr7objid'] == dr7objid]

        if len(image_data) == 0:
            print('Shit', dr7objid)

        if self.transform:
            image = self.transform(image)

        output = [image]
        output.extend(image_data.values[0].tolist())

        return tuple(output)

In [122]:
from torchvision import transforms
from torch.utils.data import random_split

dataset = GZ2Dataset(csv_file_maps='gz2_filename_mapping.csv', csv_file_data='zoo2MainSpecz.csv', root_dir='images_gz2/images/', transform=transforms.ToTensor())

trainset, testset = random_split(dataset, [round(0.8*len(dataset)), round(0.2*len(dataset))])

In [123]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [124]:
element = next(iter(trainloader))
print(element)

[tensor([[[[0.0431, 0.0353, 0.0235,  ..., 0.0235, 0.0078, 0.0000],
          [0.0588, 0.0510, 0.0392,  ..., 0.0196, 0.0039, 0.0000],
          [0.0706, 0.0627, 0.0471,  ..., 0.0078, 0.0039, 0.0000],
          ...,
          [0.0549, 0.0510, 0.0431,  ..., 0.0000, 0.0000, 0.0039],
          [0.0431, 0.0392, 0.0314,  ..., 0.0000, 0.0000, 0.0039],
          [0.0392, 0.0353, 0.0275,  ..., 0.0000, 0.0000, 0.0039]],

         [[0.0275, 0.0196, 0.0078,  ..., 0.0314, 0.0157, 0.0039],
          [0.0431, 0.0353, 0.0235,  ..., 0.0275, 0.0118, 0.0039],
          [0.0667, 0.0588, 0.0431,  ..., 0.0157, 0.0118, 0.0078],
          ...,
          [0.0627, 0.0588, 0.0510,  ..., 0.0000, 0.0000, 0.0039],
          [0.0510, 0.0471, 0.0392,  ..., 0.0000, 0.0000, 0.0039],
          [0.0471, 0.0431, 0.0353,  ..., 0.0000, 0.0000, 0.0039]],

         [[0.0157, 0.0078, 0.0000,  ..., 0.0196, 0.0039, 0.0000],
          [0.0314, 0.0235, 0.0118,  ..., 0.0157, 0.0000, 0.0000],
          [0.0510, 0.0431, 0.0275,  ..., 